### 设计一个 电子邮件 Agent
首先要将 这个 Agent 拆分成一系列 的 Nodes。接下来 描述 节点的作用 和 转移关系。最后，通过一个共享状态将各个节点连接起来，每个节点都可以从中读取和写入数据。根据之前的学习，这个共享状态 应该指的是 对话。

### 产品团队提出 以下 需求：
- 读邮件
- 分类 根据 紧急性 和 主题
- 检索相关文档与回复
- 上报复杂任务 给人类
- 根据需要安排后续跟进

### 一些可能的场景
1. Simple product question: “How do I reset my password?”
2. Bug report: “The export feature crashes when I select PDF format”
3. Urgent billing issue: “I was charged twice for my subscription!”
4. Feature request: “Can you add dark mode to the mobile app?”
5. Complex technical issue: “Our API integration fails intermittently with 504 errors”

⭐To implement an agent in LangGraph, you will usually follow the same five steps.（果真吗？接着往下看是不是文档作者在吹牛胡扯

## Step 1: Map out your workflow as discrete steps
没得说，首先需要拆分业务场景，拆分的每一步都作为一个 Node （由一个函数实现），绘制流程图

````mermaid  theme={null}
flowchart TD
    A[START] --> B[Read Email]
    B --> C[Classify Intent]

    C -.-> D[Doc Search]
    C -.-> E[Bug Track]
    C -.-> F[Human Review]

    D --> G[Draft Reply]
    E --> G
    F --> G

    G -.-> H[Human Review]
    G -.-> I[Send Reply]

    H --> J[END]
    I --> J[END]
````

每一个节点 需要做的事：
* Read Email: Extract and parse the email content
* Classify Intent: Use an LLM to categorize urgency and topic, then route to appropriate action
* Doc Search: Query your knowledge base for relevant information
* Bug Track: Create or update issue in tracking system
* Draft Reply: Generate an appropriate response
* Human Review: Escalate to human agent for approval or handling
* Send Reply: Dispatch the email response

## Step 2: Identify what each step needs to do
第二步就是规划 每一个 节点 **具体** 要做的事，对于图中的每个节点，确定它代表什么类型的操作以及它需要什么上下文才能正常工作。    
主要是 四类：
1. LLM Steps: 文本理解 、 回复决策
2. Data Steps：外部检索信息，类似数据库
3. Action Steps：外部操作，涉及 Tool 
4. User Input Steps：人类交互

## Step 3: Design your state
为所有节点 共享记忆，需要思考哪些 需要作为状态的一部分被存储起来。比如发件人信息

## Step 4: Build your nodes
A node in LangGraph is just a Python function that takes the current state and returns updates to it.

原文档还写了一些：[错误检查](https://docs.langchain.com/oss/python/langgraph/thinking-in-langgraph#handle-errors-appropriately)

## Step 5: Wire it together
连接所有节点到 一个 working graph 中。即：提供一些必要的边。 为了 实现 human-in-the-loop ，可以使用 interrupt() 实现。一些关键代码实例：

````py
# Add only the essential edges
workflow.add_edge(START, "read_email")
workflow.add_edge("read_email", "classify_intent")
workflow.add_edge("send_reply", END)

# Compile with checkpointer for persistence, in case run graph with Local_Server --> Please compile without checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
````